# 1. Data Generation

## 1.1 fundamentals

### 1.1.1 circuits and coordinates

In [1]:
import json

circuits = []
coordinates = {}

# Read circuit names from json file
circuits_json = json.load(open('tracks.json'))
for name in circuits_json:
    circuit = circuits_json[name]['grandPrix']
    circuits.append(circuit)
    coordinates[circuit] = (float(circuits_json[name]['lat']), float(circuits_json[name]['long']))
print(coordinates)

{'Bahrain Grand Prix': (26.03358, 50.51688), 'Saudi Arabian Grand Prix': (21.63697, 39.10299), 'Australian Grand Prix': (-37.85006, 144.96902), 'Chinese Grand Prix': (31.32978, 121.22633), 'European Grand Prix': (40.37291, 49.85318), 'Miami Grand Prix': (25.95687, -80.23139), 'San Marino Grand Prix': (44.34434, 11.71951), 'Monaco Grand Prix': (43.73548, 7.42128), 'Spanish Grand Prix': (41.56848, 2.2573), 'Canadian Grand Prix': (45.50164, -73.52803), 'Austrian Grand Prix': (47.22344, 14.76069), 'British Grand Prix': (52.0733, -1.01466), 'Hungarian Grand Prix': (47.58171, 19.25061), 'Belgian Grand Prix': (50.43691, 5.97204), 'Dutch Grand Prix': (52.3877, 4.54421), 'Italian Grand Prix': (45.61997, 9.28793), 'Singapore Grand Prix': (1.29143, 103.8639), 'Japanese Grand Prix': (34.84559, 136.53895), 'Qatar Grand Prix': (25.48628, 51.45289), 'United States Grand Prix': (30.13458, -97.63585), 'Mexican Grand Prix': (19.40552, -99.09256), 'Brazilian Grand Prix': (-23.70118, -46.69795), 'Las Vega

### 1.1.2 weeks

In [2]:
#Anzahl Wochen
weeks = list(range(9,47+1))
print(weeks)

corders = list(range(1,24+1))

[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]


## 1.2 parameters

### 1.2.1 Distance between Tracks

In [3]:
%pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import geopy.distance
from itertools import product

# Compute pairwise distance between tracks with geopy
dist = {(c1, c2): geopy.distance.geodesic(coordinates[c1], coordinates[c2]).km for c1, c2 in product(circuits, repeat=2)}

#print(dist)

### 1.2.2 Circuit Typ Sim Index

In [5]:
# get circuit type of every racetrack
circuitType = {}
for name in circuits_json:
    circuit = circuits_json[name]['grandPrix']
    circuitType[circuit] = (str(circuits_json[name]['circuitType']))
#print(circuitType)

# function: is circuit c1 and c2 the same circuit type?
def circuitTypeSimilarity(c1, c2):
    # check if circuits are the same
    if c1 != c2:
        if circuitType[c1] == circuitType[c2]:
            # return 1 when both tracks are street circuits or racetracks
            return 1
        else:
            return 0
    else:
        return 0

# Compute pairwise race track similarity between tracks with geopy
circuitTypeSimularity = {(c1, c2): circuitTypeSimilarity(c1, c2) for c1, c2 in product(circuits, repeat=2)}

#print(circuitTypeSimularity)

### 1.2.3 Track Speed Sim Index

The goal is to see which track in the race calendar are particular fast or slow tracks. Later in the gurobi model fast tracks (= long straights) shoud be followed by slow tracks (= many corners).

Import the historical race data

In [6]:
import pandas as pd
import numpy as np

df_circuits = pd.read_csv('historic data/circuits.csv')  
df_lap_times = pd.read_csv("historic data/lap_times.csv")
df_races = pd.read_csv("historic data/races.csv")
df_results = pd.read_csv("historic data/results.csv")

# mark empty cells as NaN
df_results = df_results.replace(r"\N",np.NaN)

Find fastet lap per race

In [7]:
## Filter each race for the fastest avg. lapspeed (km/h)
# kill rows wit NaN values
df_results_min = df_results[df_results['fastestLapSpeed'].notna()]

# get fastest lapspeed per race
df_winner_results = df_results_min.groupby("raceId")["fastestLapSpeed"].max().reset_index()

df_winner_results.head()

,raceId,fastestLapSpeed
0,1,217.668
1,2,206.483
2,3,174.289
3,4,206.049
4,5,202.484


Enrich fastet laps with year and circuitId

In [8]:
df_circuit_results = df_winner_results.merge(df_races, on="raceId", how="left")

# rename columns
df_circuit_results = df_circuit_results[["raceId", "fastestLapSpeed", "year", "circuitId", "name"]]

df_circuit_results.head()

,raceId,fastestLapSpeed,year,circuitId,name
0,1,217.668,2009,1,Australian Grand Prix
1,2,206.483,2009,2,Malaysian Grand Prix
2,3,174.289,2009,17,Chinese Grand Prix
3,4,206.049,2009,3,Bahrain Grand Prix
4,5,202.484,2009,4,Spanish Grand Prix


Get average lapspeed per year

In [9]:
#convert fastestLapSpeed row from object to float
df_circuit_results['fastestLapSpeed'] = pd.to_numeric(df_circuit_results['fastestLapSpeed'])

# get mean speed per year
df_fastet_lap_per_year = df_circuit_results.groupby(["year"])["fastestLapSpeed"].mean().reset_index()

#rename column
df_fastet_lap_per_year = df_fastet_lap_per_year.rename(columns={'fastestLapSpeed': 'avgLapSpeedSeason'})

df_fastet_lap_per_year.head(6)

,year,avgLapSpeedSeason
0,2004,217.046278
1,2005,214.745316
2,2006,210.956333
3,2007,202.681000
4,2008,205.950222
5,2009,205.283176


Enrich fastest lap per year with avg speed and get a speed index

In [10]:
# add avg speed per season to the fastest laps per race
df_circuit_results_with_avg = df_circuit_results.merge(df_fastet_lap_per_year, on="year", how="left")

# get speed index 1: avg, < 1: slower, >1: faster
df_circuit_results_with_avg["speedIndex"] = df_circuit_results_with_avg["fastestLapSpeed"]/df_circuit_results_with_avg["avgLapSpeedSeason"]

newdf = df_circuit_results_with_avg[(df_circuit_results_with_avg.year == 2021)]
newdf.head(25)

,raceId,fastestLapSpeed,year,circuitId,name,avgLapSpeedSeason,speedIndex
322,1051,232.799,2021,78,Qatar Grand Prix,217.707429,1.069320
323,1052,211.566,2021,3,Bahrain Grand Prix,217.707429,0.971790
324,1053,230.403,2021,21,Emilia Romagna Grand Prix,217.707429,1.058315
325,1054,209.738,2021,75,Portuguese Grand Prix,217.707429,0.963394
326,1055,215.357,2021,4,Spanish Grand Prix,217.707429,0.989204
327,1056,164.769,2021,6,Monaco Grand Prix,217.707429,0.756837
328,1057,206.839,2021,73,Azerbaijan Grand Prix,217.707429,0.950078
329,1058,231.811,2021,70,Styrian Grand Prix,217.707429,1.064782
330,1059,218.156,2021,34,French Grand Prix,217.707429,1.002060
331,1060,234.815,2021,70,Austrian Grand Prix,217.707429,1.078581


get mean speedIndex per circuit over the year

In [11]:
# get mean speedIndex per circuit over the year
df_circuit_speed_index = df_circuit_results_with_avg.groupby(["circuitId"])["speedIndex", "fastestLapSpeed"].mean().reset_index()

#rename column
df_circuit_speed_index = df_circuit_speed_index.rename(columns={'speedIndex': 'avgSpeedIndex'})
df_circuit_speed_index.head()

/var/folders/lx/b4l4zn_n5pd9_9fnw5hbfxvc0000gn/T/ipykernel_15417/3043474867.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_circuit_speed_index = df_circuit_results_with_avg.groupby(["circuitId"])["speedIndex", "fastestLapSpeed"].mean().reset_index()


,circuitId,avgSpeedIndex,fastestLapSpeed
0,1,1.058351,219.037588
1,2,0.995431,204.629500
2,3,0.992857,208.011579
3,4,0.977407,203.787684
4,5,1.035114,216.389778


Enrich with circuit name

In [12]:
# inner merge with the circuitId and name
df_circuit_speed_index_names = pd.merge(df_circuit_speed_index, df_circuit_results_with_avg[["circuitId", "name"]], on="circuitId", how="inner")

# drop duplicates
df_circuit_speed_index_names.drop_duplicates(keep="first", inplace=True, subset=["circuitId"])

df_circuit_speed_index_names.head()

,circuitId,avgSpeedIndex,fastestLapSpeed,name
0,1,1.058351,219.037588,Australian Grand Prix
17,2,0.995431,204.629500,Malaysian Grand Prix
31,3,0.992857,208.011579,Bahrain Grand Prix
50,4,0.977407,203.787684,Spanish Grand Prix
69,5,1.035114,216.389778,Turkish Grand Prix


remove every circuit that is not in the 2023 calendar

In [13]:
df_circuit_speed_index_names_23 = df_circuit_speed_index_names[(df_circuit_speed_index_names["circuitId"] == 3) | (df_circuit_speed_index_names["circuitId"] == 77) | (df_circuit_speed_index_names["circuitId"] == 1) | (df_circuit_speed_index_names["circuitId"] == 17) | (df_circuit_speed_index_names["circuitId"] == 73) | (df_circuit_speed_index_names["circuitId"] == 79) | (df_circuit_speed_index_names["circuitId"] == 21) | (df_circuit_speed_index_names["circuitId"] == 6) | (df_circuit_speed_index_names["circuitId"] == 4) | (df_circuit_speed_index_names["circuitId"] == 7) | (df_circuit_speed_index_names["circuitId"] == 70) | (df_circuit_speed_index_names["circuitId"] == 9) | (df_circuit_speed_index_names["circuitId"] == 11) | (df_circuit_speed_index_names["circuitId"] == 13) | (df_circuit_speed_index_names["circuitId"] == 39) | (df_circuit_speed_index_names["circuitId"] == 14) | (df_circuit_speed_index_names["circuitId"] == 15) | (df_circuit_speed_index_names["circuitId"] == 22) | (df_circuit_speed_index_names["circuitId"] == 78) | (df_circuit_speed_index_names["circuitId"] == 69) | (df_circuit_speed_index_names["circuitId"] == 32) | (df_circuit_speed_index_names["circuitId"] == 18) | (df_circuit_speed_index_names["circuitId"] == 44) | (df_circuit_speed_index_names["circuitId"] == 24)]

## las Vegas is missing
las_vegas = {'circuitId':'44', 'avgSpeedIndex':1, 'name':'Las Vegas Grand Prix'}
df_circuit_speed_index_names_23 = df_circuit_speed_index_names_23.append(las_vegas, ignore_index=True)

df_circuit_speed_index_names_23.head(25)

/var/folders/lx/b4l4zn_n5pd9_9fnw5hbfxvc0000gn/T/ipykernel_15417/3620353653.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_circuit_speed_index_names_23 = df_circuit_speed_index_names_23.append(las_vegas, ignore_index=True)


,circuitId,avgSpeedIndex,fastestLapSpeed,name
0,1,1.058351,219.037588,Australian Grand Prix
1,3,0.992857,208.011579,Bahrain Grand Prix
2,4,0.977407,203.787684,Spanish Grand Prix
3,6,0.745686,154.851111,Monaco Grand Prix
4,7,1.001719,207.366813,Canadian Grand Prix
5,9,1.096100,229.269000,British Grand Prix
6,11,0.929633,193.725316,Hungarian Grand Prix
7,13,1.116384,231.369313,Belgian Grand Prix
8,14,1.188270,247.227722,Italian Grand Prix
9,15,0.830545,170.253000,Singapore Grand Prix


compute similarity

In [14]:
# function: is circuit c1 and c2 the same circuit type?
def circuitSpeedSimilarity(c1, c2):
    # check if circuits are the same
    if c1 != c2:
        # Gett avgSpeedIndex of the named circuits
        speed1 = df_circuit_speed_index_names_23.loc[df_circuit_speed_index_names_23["name"] == c1, "avgSpeedIndex"].iloc[0]
        speed2 = df_circuit_speed_index_names_23.loc[df_circuit_speed_index_names_23["name"] == c2, "avgSpeedIndex"].iloc[0]

        #calculate difference
        speedDifference = abs(speed1 - speed2)

        # convert to simularity
        speedSimilarity = 1 - speedDifference
    else:
        speedSimilarity = 0

    return speedSimilarity

# Compute pairwise race track similarity between tracks with geopy
circuitSpeedSimularity = {(c1, c2): circuitSpeedSimilarity(c1, c2) for c1, c2 in product(circuits, repeat=2)}

#print(circuitSpeedSimularity)

### 1.2.4 Temperature &  Precipitation Index

In [15]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Import Meteostat library and dependencies
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Normals, Point
import pandas as pd

# setup base dataframe
weatherData = pd.DataFrame()

# Get Normals data for each circuit
for circuit in coordinates:
    #print(coordinates[track])

    lat = coordinates[circuit][0]
    long = coordinates[circuit][1]

    # get nearest weather station to the circuit geolocations
    Point.radius = 65000 # search for weather stations in a 65 km radius
    place = Point(lat, long)

    # Fetch data for the last 30 years
    data = Normals(place, 1991, 2020)
    data = data.fetch()

    # Plot line chart including average temperature in °C and precipitation in mm
    #data.plot(y=['prcp', 'tavg'])
    #plt.show()

    if circuit == "British Grand Prix":
        weatherData[circuit + " tavg"] = [4.4,5.2,6,9.4,12.6,15.5,17.7,17.0,14.4,11.2,7.6,5.2]

        #Calculate difference to perfect weather
        diff_arr = []
        for x in weatherData[circuit + " tavg"]:
            if x > 25:
                diff_arr.append(x - 25)
            elif x < 20:
                diff_arr.append(20 - x)
            else:
                diff_arr.append(0)
        weatherData[circuit + " tdiff"] = diff_arr

        weatherData[circuit + " prcp"] = [57,46,49,56,62,62,65,64,59,63,94,59]
    else:
        weatherData[circuit + " tavg"] = data["tavg"]

        #Calculate difference to perfect weather
        diff_arr = []
        for x in data["tavg"]:
            if x > 25:
                diff_arr.append(x - 25)
            elif x < 20:
                diff_arr.append(20 - x)
            else:
                diff_arr.append(0)
        weatherData[circuit + " tdiff"] = diff_arr

        weatherData[circuit + " prcp"] = data["prcp"]
#weatherData.to_csv("weather.csv")
weatherData.head()

,Bahrain Grand Prix tavg,Bahrain Grand Prix tdiff,Bahrain Grand Prix prcp,Saudi Arabian Grand Prix tavg,Saudi Arabian Grand Prix tdiff,Saudi Arabian Grand Prix prcp,Australian Grand Prix tavg,Australian Grand Prix tdiff,Australian Grand Prix prcp,Chinese Grand Prix tavg,...,Mexican Grand Prix prcp,Brazilian Grand Prix tavg,Brazilian Grand Prix tdiff,Brazilian Grand Prix prcp,Las Vegas Grand Prix tavg,Las Vegas Grand Prix tdiff,Las Vegas Grand Prix prcp,Abu Dhabi Grand Prix tavg,Abu Dhabi Grand Prix tdiff,Abu Dhabi Grand Prix prcp
month,,,,,,,,,,,,,,,,,,,,,
1,17.6,2.4,20.6,23.6,0.0,13.6,20.4,0.0,43.7,5.1,...,7.2,24.0,0.0,296.5,9.5,10.5,14.3,19.4,0.6,14.8
2,19.0,1.0,11.9,24.4,0.0,3.6,20.4,0.0,38.2,6.8,...,3.2,24.5,0.0,259.1,11.8,8.2,20.7,21.0,0.0,5.6
3,22.4,0.0,14.8,26.0,1.0,2.2,18.4,1.6,29.3,10.5,...,19.4,23.8,0.0,205.8,15.8,4.2,10.7,24.2,0.0,9.9
4,26.9,1.9,7.1,28.6,3.6,2.3,15.2,4.8,38.7,15.8,...,27.1,22.2,0.0,89.2,19.8,0.2,5.2,28.7,3.7,5.0
5,32.1,7.1,0.5,31.0,6.0,0.1,12.6,7.4,35.6,21.2,...,53.0,19.4,0.6,63.7,25.0,0.0,1.8,32.4,7.4,0.4


In [17]:
# format index as dateTime object
weatherDataT = weatherData.T
weatherDataT.columns = pd.to_datetime(weatherDataT.columns, format='%m')
weatherDataNT = weatherDataT.T

# stretch monthly data to weekly data
weatherDataNT = weatherDataNT.asfreq('W', method='ffill')
#weatherDataNT /= weatherDataNT.groupby(weatherDataNT.index.strftime('%Y-%m')).transform('mean')

# reset index
weatherDataNT = weatherDataNT.reset_index(level=0)
weatherDataNT['Week_Number'] = weatherDataNT['month'].dt.isocalendar().week

weatherDataNT.head()

,month,Bahrain Grand Prix tavg,Bahrain Grand Prix tdiff,Bahrain Grand Prix prcp,Saudi Arabian Grand Prix tavg,Saudi Arabian Grand Prix tdiff,Saudi Arabian Grand Prix prcp,Australian Grand Prix tavg,Australian Grand Prix tdiff,Australian Grand Prix prcp,...,Brazilian Grand Prix tavg,Brazilian Grand Prix tdiff,Brazilian Grand Prix prcp,Las Vegas Grand Prix tavg,Las Vegas Grand Prix tdiff,Las Vegas Grand Prix prcp,Abu Dhabi Grand Prix tavg,Abu Dhabi Grand Prix tdiff,Abu Dhabi Grand Prix prcp,Week_Number
0,1900-01-07,17.6,2.4,20.6,23.6,0.0,13.6,20.4,0.0,43.7,...,24.0,0.0,296.5,9.5,10.5,14.3,19.4,0.6,14.8,1
1,1900-01-14,17.6,2.4,20.6,23.6,0.0,13.6,20.4,0.0,43.7,...,24.0,0.0,296.5,9.5,10.5,14.3,19.4,0.6,14.8,2
2,1900-01-21,17.6,2.4,20.6,23.6,0.0,13.6,20.4,0.0,43.7,...,24.0,0.0,296.5,9.5,10.5,14.3,19.4,0.6,14.8,3
3,1900-01-28,17.6,2.4,20.6,23.6,0.0,13.6,20.4,0.0,43.7,...,24.0,0.0,296.5,9.5,10.5,14.3,19.4,0.6,14.8,4
4,1900-02-04,19.0,1.0,11.9,24.4,0.0,3.6,20.4,0.0,38.2,...,24.5,0.0,259.1,11.8,8.2,20.7,21.0,0.0,5.6,5


# 2. Gurobi

## 2.1 Preprocessing

### 2.1.1 normalizing

In [18]:
# Normalizing distance with max norm
factor=1.0/max(dist.values())
dist_norm = {k: v*factor for k, v in dist.items() }
#print(dist_norm)

# Circuit Type is already normalized

# Circuit Speed is already normalized

# normalizing temperature delta and percipitation
circuit_temps = []
circuit_temps_val = []
circuit_prcps = []
circuit_prcps_val = []

# get max temperature delta and max percipitation
for circuit in circuits:
    circuit_temps_val = weatherDataNT[circuit + ' tdiff'].tolist()
    circuit_temps = circuit_temps + circuit_temps_val
    
    circuit_prcps_val = weatherDataNT[circuit + ' prcp'].tolist()
    circuit_prcps = circuit_prcps + circuit_prcps_val
circuit_temp_max = max(circuit_temps)
circuit_prcp_max = max(circuit_prcps)

# calc normalized values
for circuit in circuits:
    weatherDataNT[circuit + ' tdiffN'] = weatherDataNT[circuit + ' tdiff']/circuit_temp_max
    weatherDataNT[circuit + ' prcpN'] = weatherDataNT[circuit + ' prcp']/circuit_prcp_max
weatherDataNT.head()

,month,Bahrain Grand Prix tavg,Bahrain Grand Prix tdiff,Bahrain Grand Prix prcp,Saudi Arabian Grand Prix tavg,Saudi Arabian Grand Prix tdiff,Saudi Arabian Grand Prix prcp,Australian Grand Prix tavg,Australian Grand Prix tdiff,Australian Grand Prix prcp,...,United States Grand Prix tdiffN,United States Grand Prix prcpN,Mexican Grand Prix tdiffN,Mexican Grand Prix prcpN,Brazilian Grand Prix tdiffN,Brazilian Grand Prix prcpN,Las Vegas Grand Prix tdiffN,Las Vegas Grand Prix prcpN,Abu Dhabi Grand Prix tdiffN,Abu Dhabi Grand Prix prcpN
0,1900-01-07,17.6,2.4,20.6,23.6,0.0,13.6,20.4,0.0,43.7,...,0.361314,0.234401,0.175182,0.024283,0.0,1.000000,0.383212,0.048229,0.021898,0.049916
1,1900-01-14,17.6,2.4,20.6,23.6,0.0,13.6,20.4,0.0,43.7,...,0.361314,0.234401,0.175182,0.024283,0.0,1.000000,0.383212,0.048229,0.021898,0.049916
2,1900-01-21,17.6,2.4,20.6,23.6,0.0,13.6,20.4,0.0,43.7,...,0.361314,0.234401,0.175182,0.024283,0.0,1.000000,0.383212,0.048229,0.021898,0.049916
3,1900-01-28,17.6,2.4,20.6,23.6,0.0,13.6,20.4,0.0,43.7,...,0.361314,0.234401,0.175182,0.024283,0.0,1.000000,0.383212,0.048229,0.021898,0.049916
4,1900-02-04,19.0,1.0,11.9,24.4,0.0,3.6,20.4,0.0,38.2,...,0.284672,0.140641,0.102190,0.010793,0.0,0.873862,0.299270,0.069815,0.000000,0.018887


### 2.1.2 initial weighting

In [19]:
d_w = 2.5   # 50 percent, distance d_ij
t_w = 0.75  # 15 percent, type similarity t_ij
s_w = 0.75  # 15 percent, speed similarity s_ij
c_w = 0.75  # 15 percent, temperature deviation c_iw
p_w = 0.75  # 15 percent, precipitation p_iw

## 2.2 Model

In [20]:
#pip install gurobipy
#conda install gurobi

import gurobipy as gp
from gurobipy import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print(gp.gurobi.version())

# Create environment with WLS license
#e = gp.Env(empty=True)
#e.setParam('WLSACCESSID', '6d399f2d-6f26-459d-97bc-8dabb251308f')
#e.setParam('WLSSECRET', '5f2bda76-bb31-4fb6-abc8-8f304489f75f')
#e.setParam('LICENSEID', 886330)
#e.start()

#model = gp.Model(env=e)
model = gp.Model()

(10, 0, 0)
Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-07


## 2.3 Variablen

In [21]:
# Ist das Rennen in dieser Woche geplant??
is_planned = model.addVars(circuits, weeks, vtype=GRB.BINARY, lb=0, name='x')

## 2.4 Objective

In [22]:
#model.ModelSense = GRB.MINIMIZE
obj = gp.quicksum(is_planned[circuit1, week]*is_planned[circuit2, week+1]*(d_w*dist_norm[circuit1, circuit2]+t_w*circuitTypeSimularity[circuit1, circuit2]+s_w*circuitSpeedSimularity[circuit1, circuit2])
                    +is_planned[circuit1, week]*(c_w*weatherDataNT[circuit1 +" tdiffN"].values[week-1] + p_w * weatherDataNT[circuit1 +" prcpN"].values[week-1])
                  for circuit1 in circuits for circuit2 in circuits for week in weeks[:-1])
model.setObjective(obj, GRB.MINIMIZE)

## 2.5 Constraints

In [23]:
# jedes Rennen min einmal geplant aber max eine Woche Pause (--> 2x gleiches Rennen hintereinander)
for circuit in circuits:
    model.addConstr(gp.quicksum(is_planned[circuit, week] for week in weeks) == [1,2])

# pro Woche ein Rennen
for week in weeks:
    if week in [23,31,32,33]:
        # keine Rennen in Woche 23, 31, 32, 33
        model.addConstr(gp.quicksum(is_planned[circuit, week] for circuit in circuits) == 0)
    else:
        model.addConstr(gp.quicksum(is_planned[circuit, week] for circuit in circuits) == 1)

## 2.6 Optimize

In [24]:
model.setParam('TimeLimit', 120)
model.optimize()

Set parameter TimeLimit to value 120
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 63 rows, 960 columns and 1896 nonzeros
Model fingerprint: 0xceb12944
Model has 20976 quadratic objective terms
Variable types: 24 continuous, 936 integer (936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 2e+01]
  QObjective range [9e-01, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 323.6664599
Presolve removed 4 rows and 96 columns
Presolve time: 0.16s
Presolved: 17723 rows, 18528 columns, 54696 nonzeros
Variable types: 0 continuous, 18528 integer (18528 binary)

Root relaxation: objective 1.090243e+02, 376 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept

# 3. Data Analytics

## 3.1 initial output

In [25]:
# output optimized race_calendar as dataframe
data = []
for w in weeks:
    for c in circuits:
        if int(is_planned[c, w].x) == 1:
            data_temp = [w,c]
            data.append(data_temp)
df_races = pd.DataFrame(data, columns=["Week", "Circuit"])

print(df_races)

    Week                   Circuit
0      9     Australian Grand Prix
1     10  Saudi Arabian Grand Prix
2     11  Saudi Arabian Grand Prix
3     12        Bahrain Grand Prix
4     13        Bahrain Grand Prix
5     14          Miami Grand Prix
6     15        Mexican Grand Prix
7     16        Mexican Grand Prix
8     17  United States Grand Prix
9     18      Las Vegas Grand Prix
10    19      Las Vegas Grand Prix
11    20       Austrian Grand Prix
12    21          Dutch Grand Prix
13    22        Belgian Grand Prix
14    24        Spanish Grand Prix
15    25        Spanish Grand Prix
16    26         Monaco Grand Prix
17    27     San Marino Grand Prix
18    28     San Marino Grand Prix
19    29         Monaco Grand Prix
20    30        British Grand Prix
21    34      Brazilian Grand Prix
22    35       Canadian Grand Prix
23    36       European Grand Prix
24    37       European Grand Prix
25    38      Hungarian Grand Prix
26    39      Singapore Grand Prix
27    40       Japan

## 3.2 enrich output

we create a function that takes the results and enrich the data with the coordinates of the track, circuit type and speedIndex
We also compute the overall traveling distance and tracktype- and speed similarity and also the sum of the temp- and precp-Delta.

In [26]:
def enrichResults(result):
    # prepare coordinates
    df_coordinates = pd.DataFrame.from_dict(coordinates, orient="index", columns=["Latitude","Longitude"]).reset_index().rename(columns={"index": "Circuit"})
    
    # prepare circuit type
    df_circuitType = pd.DataFrame.from_dict(circuitType, orient="index", columns=["Circuit_Type"]).reset_index().rename(columns={"index": "Circuit"})
    
    # prepare speedIndex type
    df_circuit_speed_index_renamed = df_circuit_speed_index_names_23.rename(columns={"name": "Circuit"})
    
    # get distances between two circuits
    circuit_names = []
    data_circuit_names = []
    for index, row in result.iterrows():
        # add circuit name to list
        circuit_names.append(row['Circuit'])

        # get distance
        distance = dist[circuit_names[index-1], circuit_names[index]]

        # save distance
        data_circuit_names_temp = [circuit_names[index],distance]
        data_circuit_names.append(data_circuit_names_temp)
    df_distances = pd.DataFrame(data_circuit_names, columns=["Circuit", "Distance"])
    
    #
    # get track type similarity between two circuits
    #
    
    circuit_names = []
    data_circuit_type_sim_names = []
    for index, row in result.iterrows():
        # add circuit name to list
        circuit_names.append(row['Circuit'])

        # get distance
        distance = circuitTypeSimularity[circuit_names[index-1], circuit_names[index]]

        # save distance
        data_circuit_type_sim_temp = [circuit_names[index],distance]
        data_circuit_type_sim_names.append(data_circuit_type_sim_temp)
    df_type_sim = pd.DataFrame(data_circuit_type_sim_names, columns=["Circuit", "Type Similarity"])
    
    #
    # get track speed similarity between two circuits
    #
    
    circuit_names2 = []
    data_circuit_speed_sim_names = []
    for index, row in result.iterrows():
        # add circuit name to list
        circuit_names2.append(row['Circuit'])

        # get distance
        distance = circuitSpeedSimularity[circuit_names2[index-1], circuit_names2[index]]

        # save distance
        data_circuit_speed_sim_temp = [circuit_names2[index],distance]
        data_circuit_speed_sim_names.append(data_circuit_speed_sim_temp)

    df_speed_sim = pd.DataFrame(data_circuit_speed_sim_names, columns=["Circuit", "Speed Similarity"])
    
    #
    # merge speed, track type & speed similarities to the output
    #
    
    df_races_complete = result.merge(df_coordinates, on="Circuit", how="left")
    df_races_complete = df_races_complete.merge(df_circuitType, on="Circuit", how="left")
    df_races_complete = df_races_complete.merge(df_circuit_speed_index_renamed, on="Circuit", how="left")
    df_races_complete = df_races_complete.merge(df_distances, on="Circuit", how="left").drop_duplicates(subset=["Week"], keep="first").reset_index()
    df_races_complete = df_races_complete.merge(df_type_sim, on="Circuit", how="left").drop_duplicates(subset=["index"], keep="first").reset_index().drop(['level_0'], axis=1)
    df_races_complete = df_races_complete.merge(df_speed_sim, on="Circuit", how="left").drop_duplicates(subset=["index"], keep="first").reset_index().drop(['level_0', 'index'], axis=1)

    #
    #append temperature and precipitation delta
    #
    
    temperature_list = []
    precipitation_list = []

    for index, row in df_races_complete.iterrows():
        var_week = df_races_complete["Week"].iloc[index]
        var_circuit = df_races_complete["Circuit"].iloc[index]
        temperature_list.append(weatherDataNT[var_circuit +" tdiff"].values[var_week-1])
        precipitation_list.append(weatherDataNT[var_circuit +" prcp"].values[var_week-1])

    df_races_complete["TempDelta"] = temperature_list
    df_races_complete["PrcpDelta"] = precipitation_list
    
    #
    # remove duplicate circuits based on worse values
    #
    df_races_complete["sum"] = df_races_complete["Distance"] +df_races_complete["Type Similarity"] + df_races_complete["Speed Similarity"] + df_races_complete["TempDelta"] + df_races_complete["PrcpDelta"]
    df_races_complete = df_races_complete.sort_values(by=['sum','Week'], ascending=True).groupby("Circuit").first().sort_values(by='Week', ascending=True)
    
    #
    # return
    #
    return df_races_complete

call the function

In [27]:
results_df = enrichResults(df_races)
results_df.round(2).to_csv("initial_results.csv")
results_df.head(50)

,Week,Latitude,Longitude,Circuit_Type,circuitId,avgSpeedIndex,fastestLapSpeed,Distance,Type Similarity,Speed Similarity,TempDelta,PrcpDelta,sum
Circuit,,,,,,,,,,,,,
Australian Grand Prix,9,-37.85006,144.96902,raceTrack,1,1.058351,219.037588,0.000000,0,0.000000,1.6,29.3,30.900000
Saudi Arabian Grand Prix,10,21.63697,39.10299,streetCircuit,77,1.130833,243.759000,12815.351913,0,0.927518,1.0,2.2,12819.479431
Bahrain Grand Prix,13,26.03358,50.51688,raceTrack,3,0.992857,208.011579,1259.989884,0,0.862025,1.9,7.1,1269.851908
Miami Grand Prix,14,25.95687,-80.23139,raceTrack,79,0.999189,213.255000,12204.814600,1,0.993668,0.0,72.2,12279.008268
Mexican Grand Prix,15,19.40552,-99.09256,raceTrack,32,0.930799,195.270000,2066.799487,1,0.931610,0.0,27.1,2095.831097
United States Grand Prix,17,30.13458,-97.63585,raceTrack,69,0.969487,200.537778,1197.528424,1,0.961312,0.0,57.4,1256.889735
Las Vegas Grand Prix,18,36.11511,-115.17539,streetCircuit,44,1.000000,NaN,1762.744648,0,0.969487,0.0,1.8,1765.514135
Austrian Grand Prix,20,47.22344,14.76069,raceTrack,70,1.078645,229.112636,9517.565692,0,0.921355,4.4,91.3,9614.187046
Dutch Grand Prix,21,52.38770,4.54421,raceTrack,39,0.990568,215.654000,931.686662,1,0.911923,6.5,42.7,982.798585


## 3.3 evaluate output

In [28]:
def evaluateResults(enrichedResults):
    resultsList = []

    resultsList.append(enrichedResults["Distance"].sum())
    resultsList.append(enrichedResults["Type Similarity"].sum())
    resultsList.append(enrichedResults["Speed Similarity"].sum())
    resultsList.append(enrichedResults["TempDelta"].sum())
    resultsList.append(enrichedResults["PrcpDelta"].sum())
    resultsList.append(enrichedResults["sum"].sum())

    return resultsList

call the function

In [29]:
results = evaluateResults(results_df)
    
print(results)

[101893.59940858194, 12, 20.29188657610947, 52.6, 1265.7, 103244.19129515806]


## 3.4 Map

pip install folium

In [30]:
#pip install folium

Preview

In [31]:
# Map the solution

import folium
import branca.colormap as cm
from folium import plugins

map = folium.Map(location=[20,5], zoom_start = 2)
ii = 1
points = []
max_dist = results_df["Distance"].max()

for index, location_info in results_df.iterrows():
    icon_number = plugins.BeautifyIcon(
        border_color="#8A90B4",
        text_color="#8A90B4",
        number=ii,
        iconShape="marker",
        inner_icon_style="margin-top:0;",
    )
    folium.Marker(
        [location_info["Latitude"], location_info["Longitude"]],
        popup=folium.Popup(
            "<div><b>Strecke:</b> "+index+
            "</div><div><b>Kalenderwoche:</b> "+str(location_info["Week"])+
            "</div><div><b>Distanz:</b> "+str(int(round(location_info["Distance"],0)))+
            " km</div><div><b>Typ:</b> "+location_info["Circuit_Type"]+
            "</div><div><b>Geschw. Index:</b> "+str(round(location_info["avgSpeedIndex"],2))+
            "</div><div><b>Temperaturabw.:</b> "+str(round(location_info["TempDelta"],1))+
            "°C</div><div><b>Niederschlag:</b> "+str(location_info["PrcpDelta"])+
            " mm/Monat</div>",
            parse_html=False, max_width=200),
        icon=icon_number
    ).add_to(map)

    points.append([location_info["Latitude"], location_info["Longitude"]])
    ii += 1
points.append(points[0])

#print(points)

colormap = cm.LinearColormap(["green", "yellow", "red"], vmin=3, vmax=max_dist,caption='step')
colormap.caption = "Reisedistanz (km)"
map.add_child(colormap)

for i, point in enumerate(points[:-1]):
    if i > 0:
        # get information about the route
        point_info = results_df.loc[results_df['Latitude'] == points[i][0]]
        distance = str(round(point_info["Distance"].iloc[0],0))
        #print(point_info)
        #print(points[i-1], points[i])
        route = folium.PolyLine([points[i-1], points[i]], fillColor="blue", color=colormap(int(float(distance)))).add_to(map)

### 3.4.1 Preview

In [32]:
map

# 4. Sensitivity analysis

The idea is to call the model iteratively with different weightings and check how the results are changing.

Because the problem is presumably NP-hard we need to set a timelimit.

As we can see in the first model solution the improvment of the solution gets very low after 60 seconds on a apple m1 pro 10 core cpu with gurobi 10.0.

### 4.1 Timelimit

In [33]:
model.setParam('TimeLimit', 120)

### 4.2 sensitivity analysis

In [34]:
resultsMatrix = []
resultsMatrixDistance = []
resultsMatrixTypeSim = []
resultsMatrixSpeedSim = []
resultsMatrixTemp = []
resultsMatrixPrcp = []

sensitivityParameters = range(1,5+1,1) # start, stop, steps
sensitivityRange = range(0,25+1,1) # start, stop, steps

for x in sensitivityParameters:
    for y in sensitivityRange:
        # set weighting
        if x == 1:
            d_w = y/26*5         # x5 to make shure the sum of weights is 5
            t_w = (26-y)/26*1.25 # x1.25 to make shure the sum of weights is 5
            s_w = (26-y)/26*1.25
            c_w = (26-y)/26*1.25
            p_w = (26-y)/26*1.25
        elif x == 2:
            d_w = (26-y)/26*1.25
            t_w = y/26*5
            s_w = (26-y)/26*1.25
            c_w = (26-y)/26*1.25
            p_w = (26-y)/26*1.25
        elif x == 3:
            d_w = (26-y)/26*1.25
            t_w = (26-y)/26*1.25
            s_w = y/26*5
            c_w = (26-y)/26*1.25
            p_w = (26-y)/26*1.25
        elif x == 4:
            d_w = (26-y)/26*1.25
            t_w = (26-y)/26*1.25
            s_w = (26-y)/26*1.25
            c_w = y/26*5
            p_w = (26-y)/26*1.25
        elif x == 5:
            d_w = (26-y)/26*1.25
            t_w = (26-y)/26*1.25
            s_w = (26-y)/26*1.25
            c_w = (26-y)/26*1.25
            p_w = y/26*5
        # set objective
        #model.ModelSense = GRB.MINIMIZE
        obj = gp.quicksum(is_planned[circuit1, week]*is_planned[circuit2, week+1]*(d_w*dist_norm[circuit1, circuit2]+t_w*circuitTypeSimularity[circuit1, circuit2]+s_w*circuitSpeedSimularity[circuit1, circuit2])
                            +is_planned[circuit1, week]*(c_w*weatherDataNT[circuit1 +" tdiffN"].values[week-1] + p_w * weatherDataNT[circuit1 +" prcpN"].values[week-1])
                          for circuit1 in circuits for circuit2 in circuits for week in weeks[:-1])
        model.setObjective(obj, GRB.MINIMIZE)

        # reset and run model
        model.reset(0)
        model.optimize()

        # output optimized race_calendar as dataframe
        data = []
        for w in weeks:
            for c in circuits:
                if int(is_planned[c, w].x) == 1:
                    data_temp = [w,c]
                    data.append(data_temp)
        df_races = pd.DataFrame(data, columns=["Week", "Circuit"])

        # enrich result
        results_df = enrichResults(df_races)

        # get the results
        results = evaluateResults(results_df)
        print(results)
        if x == 1:
            resultsMatrixDistance.append(results[0])
        elif x == 2:
            resultsMatrixTypeSim.append(results[1])
        elif x == 3:
            resultsMatrixSpeedSim.append(results[2])
        elif x == 4:
            resultsMatrixTemp.append(results[3])
        elif x == 5:
            resultsMatrixPrcp.append(results[4])
        
data = {'Distance': resultsMatrixDistance,'TypeSim': resultsMatrixTypeSim, 'SpeedSim': resultsMatrixSpeedSim, 'Temp': resultsMatrixTemp,'Prcp': resultsMatrixPrcp}
sens_results = pd.DataFrame(data)

Discarded solution information
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 63 rows, 960 columns and 1896 nonzeros
Model fingerprint: 0xa0c2d67b
Model has 20976 quadratic objective terms
Variable types: 24 continuous, 936 integer (936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 3e+01]
  QObjective range [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 485.7243075
Presolve removed 4 rows and 96 columns
Presolve time: 0.15s
Presolved: 17723 rows, 18528 columns, 54696 nonzeros
Variable types: 0 continuous, 18528 integer (18528 binary)

Root relaxation: objective 1.817072e+02, 393 iterations, 0.02 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI

In [35]:
resultsMatrixDistance_norm = [float(i)/max(resultsMatrixDistance) for i in resultsMatrixDistance]
resultsMatrixTypeSim_norm = [float(i)/max(resultsMatrixTypeSim) for i in resultsMatrixTypeSim]
resultsMatrixSpeedSim_norm = [float(i)/max(resultsMatrixSpeedSim) for i in resultsMatrixSpeedSim]
resultsMatrixTemp_norm = [float(i)/max(resultsMatrixTemp) for i in resultsMatrixTemp]
resultsMatrixPrcp_norm = [float(i)/max(resultsMatrixPrcp) for i in resultsMatrixPrcp]
weighting = range(0,100+1, 4)

data_norm = {'Weighting': weighting,'Distance': resultsMatrixDistance_norm,'TypeSim': resultsMatrixTypeSim_norm, 'SpeedSim': resultsMatrixSpeedSim_norm, 'Temp': resultsMatrixTemp_norm,'Prcp': resultsMatrixPrcp_norm}

sens_results = pd.DataFrame(data_norm)
sens_results.to_csv('sens_norm_results.csv')
sens_results.head(30)

,Weighting,Distance,TypeSim,SpeedSim,Temp,Prcp
0,0,1.000000,1.000000,1.000000,1.000000,1.000000
1,4,0.773880,1.000000,0.984266,0.668740,0.907078
2,8,0.791816,0.866667,0.988377,0.545101,0.867288
3,12,0.657503,0.866667,0.988377,0.456454,0.816112
4,16,0.657503,0.800000,0.982051,0.409020,0.776954
5,20,0.657503,0.800000,0.982051,0.409020,0.727790
6,24,0.657503,0.800000,0.982051,0.356921,0.727790
7,28,0.657503,0.733333,0.982051,0.282271,0.668392
8,32,0.656887,0.600000,0.982051,0.282271,0.668392
9,36,0.656887,0.600000,0.987503,0.262830,0.659364


### 4.3 second sensitivity analysis

In [36]:
model.setParam('TimeLimit', 420)

Set parameter TimeLimit to value 420


In [37]:
resultsMatrix = []
resultsMatrixDistance = []
resultsMatrixTypeSim = []
resultsMatrixSpeedSim = []
resultsMatrixTemp = []
resultsMatrixPrcp = []

sensitivity2Range = range(0,14+1,1) # start, stop, steps

for y in sensitivity2Range:
    # set weighting
    d_w = (y*0.025 + 0.5)*5
    t_w = 0.05*5
    s_w = 0.025*5
    c_w = (15-y)*0.025*5
    p_w = 0.05*5
    
    # set objective
    #model.ModelSense = GRB.MINIMIZE
    obj = gp.quicksum(is_planned[circuit1, week]*is_planned[circuit2, week+1]*(d_w*dist_norm[circuit1, circuit2]+t_w*circuitTypeSimularity[circuit1, circuit2]+s_w*circuitSpeedSimularity[circuit1, circuit2])
                        +is_planned[circuit1, week]*(c_w*weatherDataNT[circuit1 +" tdiffN"].values[week-1] + p_w * weatherDataNT[circuit1 +" prcpN"].values[week-1])
                      for circuit1 in circuits for circuit2 in circuits for week in weeks[:-1])
    model.setObjective(obj, GRB.MINIMIZE)

    # reset and run model
    model.reset(0)
    model.optimize()

    # output optimized race_calendar as dataframe
    data = []
    for w in weeks:
        for c in circuits:
            if int(is_planned[c, w].x) == 1:
                data_temp = [w,c]
                data.append(data_temp)
    df_races = pd.DataFrame(data, columns=["Week", "Circuit"])

    # enrich result
    results_df = enrichResults(df_races)

    # get the results
    results = evaluateResults(results_df)
    print(results)
    resultsMatrixDistance.append(results[0])
    resultsMatrixTypeSim.append(results[1])
    resultsMatrixSpeedSim.append(results[2])
    resultsMatrixTemp.append(results[3])
    resultsMatrixPrcp.append(results[4])
        
data = {'Distance': resultsMatrixDistance,'TypeSim': resultsMatrixTypeSim, 'SpeedSim': resultsMatrixSpeedSim, 'Temp': resultsMatrixTemp,'Prcp': resultsMatrixPrcp}
sens_results = pd.DataFrame(data)

Discarded solution information
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 63 rows, 960 columns and 1896 nonzeros
Model fingerprint: 0xe45a1e78
Model has 20976 quadratic objective terms
Variable types: 24 continuous, 936 integer (936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 4e+01]
  QObjective range [2e-01, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 337.7312276
Presolve removed 4 rows and 96 columns
Presolve time: 0.15s
Presolved: 17723 rows, 18528 columns, 54696 nonzeros
Variable types: 0 continuous, 18528 integer (18528 binary)

Root relaxation: objective 6.007743e+01, 280 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI

In [38]:

weighting = range(0,14+1,1)

data2 = {'Weighting': weighting,'Distance': resultsMatrixDistance,'TypeSim': resultsMatrixTypeSim, 'SpeedSim': resultsMatrixSpeedSim, 'Temp': resultsMatrixTemp,'Prcp': resultsMatrixPrcp}
print(data2)
sens_results = pd.DataFrame(data2)
sens_results.to_csv('sens_sec_results.csv')
sens_results.head(30)

{'Weighting': range(0, 15), 'Distance': [77826.25043748695, 77826.25043748695, 77826.25043748695, 77826.25043748695, 77826.25043748695, 77826.25043748695, 76529.98110915774, 76529.98110915774, 76529.98110915774, 76494.71766485323, 75965.3120720276, 75965.3120720276, 74041.48737842697, 79164.43404571626, 72688.67804396892], 'TypeSim': [12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13], 'SpeedSim': [20.578616444984466, 20.578616444984466, 20.578616444984466, 20.578616444984466, 20.578616444984466, 20.578616444984466, 20.529990610588108, 20.529990610588108, 20.529990610588108, 20.167866223804594, 20.419836299548628, 20.419836299548628, 20.988108446630026, 20.597923574365247, 20.810251107612757], 'Temp': [28.900000000000002, 28.900000000000002, 28.900000000000002, 28.900000000000002, 28.900000000000002, 28.900000000000002, 29.3, 29.3, 29.3, 29.8, 32.6, 32.6, 30.60000000000001, 37.400000000000006, 71.9], 'Prcp': [1582.3000000000002, 1582.3000000000002, 1582.3000000000002, 1582.300

,Weighting,Distance,TypeSim,SpeedSim,Temp,Prcp
0,0,77826.250437,12,20.578616,28.9,1582.3
1,1,77826.250437,12,20.578616,28.9,1582.3
2,2,77826.250437,12,20.578616,28.9,1582.3
3,3,77826.250437,12,20.578616,28.9,1582.3
4,4,77826.250437,12,20.578616,28.9,1582.3
5,5,77826.250437,12,20.578616,28.9,1582.3
6,6,76529.981109,12,20.529991,29.3,1564.5
7,7,76529.981109,12,20.529991,29.3,1564.5
8,8,76529.981109,12,20.529991,29.3,1564.5
9,9,76494.717665,12,20.167866,29.8,1559.0


Was sind die Werte des offiziellen F1 Kalenders für die Saison 2023?

In [39]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

resultsMatrixDistance_norm = NormalizeData(resultsMatrixDistance)
resultsMatrixTypeSim_norm = NormalizeData(resultsMatrixTypeSim)
resultsMatrixSpeedSim_norm = NormalizeData(resultsMatrixSpeedSim)
resultsMatrixTemp_norm = NormalizeData(resultsMatrixTemp)
resultsMatrixPrcp_norm = NormalizeData(resultsMatrixPrcp)
w_d = np.arange(50, 85+2.5, 2.5)
w_c = np.arange(37.5, 2.5-2.5, -2.5)

data_norm = {'w_d': w_d,'w_c': w_c,'Distance': resultsMatrixDistance_norm,'TypeSim': resultsMatrixTypeSim_norm, 'SpeedSim': resultsMatrixSpeedSim_norm, 'Temp': resultsMatrixTemp_norm,'Prcp': resultsMatrixPrcp_norm}

sens_results = pd.DataFrame(data_norm)
sens_results.to_csv('sens_sec_norm_results.csv')
sens_results.head(30)

,w_d,w_c,Distance,TypeSim,SpeedSim,Temp,Prcp
0,50.0,37.5,0.793355,0.0,0.500767,0.000000,1.000000
1,52.5,35.0,0.793355,0.0,0.500767,0.000000,1.000000
2,55.0,32.5,0.793355,0.0,0.500767,0.000000,1.000000
3,57.5,30.0,0.793355,0.0,0.500767,0.000000,1.000000
4,60.0,27.5,0.793355,0.0,0.500767,0.000000,1.000000
5,62.5,25.0,0.793355,0.0,0.500767,0.000000,1.000000
6,65.0,22.5,0.593182,0.0,0.441485,0.009302,0.951525
7,67.5,20.0,0.593182,0.0,0.441485,0.009302,0.951525
8,70.0,17.5,0.593182,0.0,0.441485,0.009302,0.951525
9,72.5,15.0,0.587737,0.0,0.000000,0.020930,0.936547


# 5. Final Model

## 5.1 Optimize

In [40]:
model.setParam('TimeLimit', 600)

d_w = 0.8*5
t_w = 0.05*5
s_w = 0.025*5
c_w = 0.075*5
p_w = 0.05*5

# set objective
#model.ModelSense = GRB.MINIMIZE
obj = gp.quicksum(is_planned[circuit1, week]*is_planned[circuit2, week+1]*(d_w*dist_norm[circuit1, circuit2]+t_w*circuitTypeSimularity[circuit1, circuit2]+s_w*circuitSpeedSimularity[circuit1, circuit2])
                    +is_planned[circuit1, week]*(c_w*weatherDataNT[circuit1 +" tdiffN"].values[week-1] + p_w * weatherDataNT[circuit1 +" prcpN"].values[week-1])
                  for circuit1 in circuits for circuit2 in circuits for week in weeks[:-1])
model.setObjective(obj, GRB.MINIMIZE)

# reset and run model
model.reset(0)
model.optimize()

# output optimized race_calendar as dataframe
data = []
for w in weeks:
    for c in circuits:
        if int(is_planned[c, w].x) == 1:
            data_temp = [w,c]
            data.append(data_temp)
df_races = pd.DataFrame(data, columns=["Week", "Circuit"])

# enrich result
results_df = enrichResults(df_races)

# get the results
results = evaluateResults(results_df)

Set parameter TimeLimit to value 600
Discarded solution information
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 63 rows, 960 columns and 1896 nonzeros
Model fingerprint: 0x79dfe3af
Model has 20976 quadratic objective terms
Variable types: 24 continuous, 936 integer (936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 9e+00]
  QObjective range [2e-01, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 161.4353164
Presolve removed 4 rows and 96 columns
Presolve time: 0.17s
Presolved: 17723 rows, 18528 columns, 54696 nonzeros
Variable types: 0 continuous, 18528 integer (18528 binary)

Root relaxation: objective 3.987029e+01, 332 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |    

## 5.2 Results

In [41]:
# Show results as table
results_df.head(26)

,Week,Latitude,Longitude,Circuit_Type,circuitId,avgSpeedIndex,fastestLapSpeed,Distance,Type Similarity,Speed Similarity,TempDelta,PrcpDelta,sum
Circuit,,,,,,,,,,,,,
Australian Grand Prix,9,-37.85006,144.96902,raceTrack,1,1.058351,219.037588,0.000000,0,0.000000,1.6,29.3,30.900000
Qatar Grand Prix,10,25.48628,51.45289,raceTrack,78,1.069320,232.799000,11993.167448,1,0.989031,0.0,19.0,12014.156478
Saudi Arabian Grand Prix,12,21.63697,39.10299,streetCircuit,77,1.130833,243.759000,1330.244628,0,0.938488,1.0,2.2,1334.383116
Brazilian Grand Prix,14,-23.70118,-46.69795,raceTrack,18,1.015750,210.472765,10553.745620,0,0.884917,0.0,89.2,10643.830536
Miami Grand Prix,15,25.95687,-80.23139,raceTrack,79,0.999189,213.255000,6575.113877,1,0.983439,0.0,72.2,6649.297317
United States Grand Prix,16,30.13458,-97.63585,raceTrack,69,0.969487,200.537778,1770.643143,1,0.970298,0.0,57.4,1830.013441
Mexican Grand Prix,18,19.40552,-99.09256,raceTrack,32,0.930799,195.270000,1197.528424,1,0.961312,0.0,53.0,1252.489735
Las Vegas Grand Prix,20,36.11511,-115.17539,streetCircuit,44,1.000000,NaN,2430.265440,0,0.930799,0.0,1.8,2432.996239
Canadian Grand Prix,22,45.50164,-73.52803,raceTrack,7,1.001719,207.366813,3620.565737,0,0.998281,0.8,106.6,3728.964017


In [42]:
# Save as csv
results_df.round(2).to_csv("final_results.csv")
# Calculate values
print(results) # distance, typeSim, SpeedSim, Temp, Prcp, Sum

[74041.48737842697, 13, 20.988108446630026, 30.60000000000001, 1549.2, 75655.27548687361]


## 5.3 Map

In [43]:
map = folium.Map(location=[20,5], zoom_start = 2)
ii = 1
points = []
max_dist = results_df["Distance"].max()

for index, location_info in results_df.iterrows():
    icon_number = plugins.BeautifyIcon(
        border_color="#8A90B4",
        text_color="#8A90B4",
        number=ii,
        iconShape="marker",
        inner_icon_style="margin-top:0;",
    )
    folium.Marker(
        [location_info["Latitude"], location_info["Longitude"]],
        popup=folium.Popup(
            "<div><b>Strecke:</b> "+index+
            "</div><div><b>Kalenderwoche:</b> "+str(location_info["Week"])+
            "</div><div><b>Distanz:</b> "+str(int(round(location_info["Distance"],0)))+
            " km</div><div><b>Typ:</b> "+location_info["Circuit_Type"]+
            "</div><div><b>Geschw. Index:</b> "+str(round(location_info["avgSpeedIndex"],2))+
            "</div><div><b>Temperaturabw.:</b> "+str(round(location_info["TempDelta"],1))+
            "°C</div><div><b>Niederschlag:</b> "+str(location_info["PrcpDelta"])+
            " mm/Monat</div>",
            parse_html=False, max_width=200),
        icon=icon_number
    ).add_to(map)

    points.append([location_info["Latitude"], location_info["Longitude"]])
    ii += 1
points.append(points[0])

#print(points)

colormap = cm.LinearColormap(["green", "yellow", "red"], vmin=3, vmax=max_dist,caption='step')
colormap.caption = "Reisedistanz (km)"
map.add_child(colormap)

for i, point in enumerate(points[:-1]):
    if i > 0:
        # get information about the route
        point_info = results_df.loc[results_df['Latitude'] == points[i][0]]
        distance = str(round(point_info["Distance"].iloc[0],0))
        #print(point_info)
        #print(points[i-1], points[i])
        route = folium.PolyLine([points[i-1], points[i]], fillColor="blue", color=colormap(int(float(distance)))).add_to(map)

### 5.3.1 Final Results

In [44]:
map

# 6. Official calendar

In [45]:
# read official calendar
df_official_calendar = pd.read_csv('fia_2023_calendar.csv', delimiter =";")
df_official_calendar.head(50)

,Week,Circuit
0,9,Bahrain Grand Prix
1,10,Bahrain Grand Prix
2,11,Saudi Arabian Grand Prix
3,12,Saudi Arabian Grand Prix
4,13,Australian Grand Prix
5,14,Australian Grand Prix
6,15,Chinese Grand Prix
7,16,Chinese Grand Prix
8,17,European Grand Prix
9,18,Miami Grand Prix


In [46]:
of_results_df = enrichResults(df_official_calendar)
of_results = evaluateResults(of_results_df)
print(of_results)

[133676.2581141122, 13, 20.43866831753313, 53.2, 1864.0, 135626.8967824297]


In [47]:
map = folium.Map(location=[20,5], zoom_start = 2)
ii = 1
points = []
max_dist = of_results_df["Distance"].max()

for index, location_info in of_results_df.iterrows():
    icon_number = plugins.BeautifyIcon(
        border_color="#8A90B4",
        text_color="#8A90B4",
        number=ii,
        iconShape="marker",
        inner_icon_style="margin-top:0;",
    )
    folium.Marker(
        [location_info["Latitude"], location_info["Longitude"]],
        popup=folium.Popup(
            "<div><b>Strecke:</b> "+index+
            "</div><div><b>Kalenderwoche:</b> "+str(location_info["Week"])+
            "</div><div><b>Distanz:</b> "+str(int(round(location_info["Distance"],0)))+
            " km</div><div><b>Typ:</b> "+location_info["Circuit_Type"]+
            "</div><div><b>Geschw. Index:</b> "+str(round(location_info["avgSpeedIndex"],2))+
            "</div><div><b>Temperaturabw.:</b> "+str(round(location_info["TempDelta"],1))+
            "°C</div><div><b>Niederschlag:</b> "+str(location_info["PrcpDelta"])+
            " mm/Monat</div>",
            parse_html=False, max_width=200),
        icon=icon_number
    ).add_to(map)

    points.append([location_info["Latitude"], location_info["Longitude"]])
    ii += 1
points.append(points[0])

#print(points)

colormap = cm.LinearColormap(["green", "yellow", "red"], vmin=3, vmax=max_dist,caption='step')
colormap.caption = "Reisedistanz (km)"
map.add_child(colormap)

for i, point in enumerate(points[:-1]):
    if i > 0:
        # get information about the route
        point_info = of_results_df.loc[of_results_df['Latitude'] == points[i][0]]
        distance = str(round(point_info["Distance"].iloc[0],0))
        #print(point_info)
        #print(points[i-1], points[i])
        route = folium.PolyLine([points[i-1], points[i]], fillColor="blue", color=colormap(int(float(distance)))).add_to(map)

## 6.1 Official map

In [48]:
map